# Setup environment and load data

Dataset: https://s3.hothienlac.com/yomitoon/sales_data.csv

In [52]:
import pandas as pd

df_raw = pd.read_csv('sales_data.csv')
df_raw

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method
0,1001,2024-01-02,C001,Alice,New York,Laptop,Electronics,1,1200,Credit Card
1,1002,2024-01-02,C002,Bob,Los Angeles,Headphones,Electronics,2,150,PayPal
2,1003,2024-01-03,C003,Charlie,New York,Office Chair,Furniture,1,350,Credit Card
3,1004,2024-01-03,C001,Alice,New York,Mouse,Electronics,3,25,Debit Card
4,1005,2024-01-04,C004,Diana,Chicago,Desk,Furniture,1,500,Bank Transfer
5,1006,2024-01-04,C005,Eve,Chicago,Laptop,Electronics,1,1100,Credit Card
6,1007,2024-01-05,C002,Bob,Los Angeles,Monitor,Electronics,2,300,Debit Card
7,1008,2024-01-05,C003,Charlie,New York,Desk Lamp,Furniture,2,45,PayPal
8,1009,2024-01-06,C006,Frank,Miami,Tablet,Electronics,1,600,Credit Card
9,1010,2024-01-06,C001,Alice,New York,Keyboard,Electronics,1,80,Bank Transfer


In [53]:
df_raw

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method
0,1001,2024-01-02,C001,Alice,New York,Laptop,Electronics,1,1200,Credit Card
1,1002,2024-01-02,C002,Bob,Los Angeles,Headphones,Electronics,2,150,PayPal
2,1003,2024-01-03,C003,Charlie,New York,Office Chair,Furniture,1,350,Credit Card
3,1004,2024-01-03,C001,Alice,New York,Mouse,Electronics,3,25,Debit Card
4,1005,2024-01-04,C004,Diana,Chicago,Desk,Furniture,1,500,Bank Transfer
5,1006,2024-01-04,C005,Eve,Chicago,Laptop,Electronics,1,1100,Credit Card
6,1007,2024-01-05,C002,Bob,Los Angeles,Monitor,Electronics,2,300,Debit Card
7,1008,2024-01-05,C003,Charlie,New York,Desk Lamp,Furniture,2,45,PayPal
8,1009,2024-01-06,C006,Frank,Miami,Tablet,Electronics,1,600,Credit Card
9,1010,2024-01-06,C001,Alice,New York,Keyboard,Electronics,1,80,Bank Transfer


In [54]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        10 non-null     int64 
 1   order_date      10 non-null     object
 2   customer_id     10 non-null     object
 3   customer_name   10 non-null     object
 4   city            10 non-null     object
 5   product         10 non-null     object
 6   category        10 non-null     object
 7   quantity        10 non-null     int64 
 8   unit_price      10 non-null     int64 
 9   payment_method  10 non-null     object
dtypes: int64(3), object(7)
memory usage: 932.0+ bytes


# 🟡 LEVEL 4 — Analytical Aggregation (Score 4–6)

## **Q8. Average order value (AOV) per customer**

### Task

For each customer, compute:

* total spending
* number of orders
* **average order value**

### 💡 Hint

Use:

* `.groupby()`
* `.agg()`
* basic arithmetic between aggregated columns

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
* [https://pandas.pydata.org/docs/user_guide/groupby.html](https://pandas.pydata.org/docs/user_guide/groupby.html)

### 🧠 Explanation

This teaches:

* multi-metric aggregation
* **ratio metrics** (very common in dashboards)
* separating *raw data* from *business KPIs*

In [55]:
df_raw['total_amount'] = df_raw['quantity'] * df_raw['unit_price']

df_q8 = df_raw.groupby('customer_id').agg(total_spending=('total_amount', 'sum'),number_of_orders=('order_id', 'nunique'))

df_q8['average_order_value'] = df_q8['total_spending'] / df_q8['number_of_orders']

In [56]:
df_q8

,total_spending,number_of_orders,average_order_value
customer_id,,,
C001,1355,3,451.666667
C002,900,2,450.000000
C003,440,2,220.000000
C004,500,1,500.000000
C005,1100,1,1100.000000
C006,600,1,600.000000


## **Q9. Revenue contribution by category (%)**

### Task

Calculate:

* total revenue per product category
* percentage contribution of each category to total revenue

### 💡 Hint

Use:

* `.groupby()`
* `.sum()`
* `.assign()`
* division by a **global scalar**

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.div.html)

### 🧠 Explanation

You learn:

* **normalization**
* how to compare groups on the same scale
* how to prepare data for **pie charts / stacked bars**

In [57]:
df_q9 = df_raw.groupby('category').agg(total_revenue=('total_amount', 'sum'))
df_q9['revenue_contribution'] = df_q9['total_revenue'] / df_q9['total_revenue'].sum() * 100

In [58]:
df_q9

,total_revenue,revenue_contribution
category,,
Electronics,3955,80.796731
Furniture,940,19.203269


# 🔵 LEVEL 5 — Distribution, Ranking & Segmentation (Score 7–8)

## **Q10. Identify top 20% customers by revenue (Pareto analysis)**

### Task

Determine:

* which customers belong to the **top 20%** by total spending

### 💡 Hint

Use:

* `.sort_values()`
* `.cumsum()`
* `.quantile()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.quantile.html)

### 🧠 Explanation

This is a classic **80/20 rule** problem:

* who really drives revenue?
* foundational for **customer segmentation**

In [59]:
df_q10 = df_raw.groupby('customer_id').agg(total_spending=('total_amount', 'sum')).sort_values('total_spending', ascending=False)

df_q10

,total_spending
customer_id,
C001,1355
C005,1100
C002,900
C006,600
C004,500
C003,440


## **Q11. Price distribution analysis per category**

### Task

For each product category, compute:

* mean unit price
* median unit price
* standard deviation

### 💡 Hint

Use:

* `.groupby()`
* `.agg(mean=..., median=..., std=...)`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

This builds intuition for:

* **distribution shape**
* why **median ≠ mean**
* choosing the right chart (boxplot vs bar)

In [60]:

df_q11 = df_raw.groupby('category').agg(mean = ('unit_price', 'mean'), median = ('unit_price', 'median'), std = ('unit_price', 'std'))
df_q11

,mean,median,std
category,,,
Electronics,493.571429,300.0,487.209010
Furniture,298.333333,350.0,231.858434


# 🔴 LEVEL 6 — Time Series, Growth & Insight (Score 9–10)

## **Q12. Day-over-day revenue growth (%)**

### Task

Compute:

* daily revenue
* **percentage change compared to previous day**

### 💡 Hint

Use:

* `.groupby()`
* `.pct_change()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.pct_change.html)

### 🧠 Explanation

This teaches:

* growth vs absolute value
* preparing data for **line charts**
* understanding volatility

In [61]:
df_q12 = df_raw.groupby('order_date').agg(daily_revenue=('total_amount', 'sum'))
df_q12['daily_revenue_growth'] = df_q12['daily_revenue'].pct_change()
df_q12

,daily_revenue,daily_revenue_growth
order_date,,
2024-01-02,1500,NaN
2024-01-03,425,-0.716667
2024-01-04,1600,2.764706
2024-01-05,690,-0.568750
2024-01-06,680,-0.014493


## **Q13. Rolling average of daily revenue**

### Task

Calculate:

* 3-day rolling average of daily revenue

### 💡 Hint

Use:

* `.rolling(window=3)`
* `.mean()`

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.rolling.html)

### 🧠 Explanation

Rolling metrics are used to:

* smooth noisy data
* reveal trends
* support **time-series visualization**

In [62]:
df_q13 = df_raw.groupby('order_date').agg(daily_revenue=('total_amount', 'sum'))
df_q13 = df_q13.rolling(window=3).mean()
df_q13

,daily_revenue
order_date,
2024-01-02,NaN
2024-01-03,NaN
2024-01-04,1175.0
2024-01-05,905.0
2024-01-06,990.0


## **Q14. Detect unusually large orders (outliers)**

### Task

Flag orders where:

* `total_amount` is significantly higher than normal
  (use a statistical threshold)

### 💡 Hint

Use:

* `.mean()`
* `.std()`
* boolean conditions

### 📚 Reference

* [https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html)

### 🧠 Explanation

You are learning:

* **basic anomaly detection**
* how math supports intuition
* how analysts decide what deserves investigation

In [63]:
df_14 = df_raw.copy()
df_14['total_amount'] = df_14['quantity'] * df_14['unit_price']
df_14['mean_total_amount'] = df_14['total_amount'].mean()
df_14['std_total_amount'] = df_14['total_amount'].std()
df_14['is_outlier'] = (df_14['total_amount'] > df_14['mean_total_amount'] + 3 * df_14['std_total_amount']) | (df_14['total_amount'] < df_14['mean_total_amount'] - 3 * df_14['std_total_amount'])
df_14

,order_id,order_date,customer_id,customer_name,city,product,category,quantity,unit_price,payment_method,total_amount,mean_total_amount,std_total_amount,is_outlier
0,1001,2024-01-02,C001,Alice,New York,Laptop,Electronics,1,1200,Credit Card,1200,489.5,402.288247,False
1,1002,2024-01-02,C002,Bob,Los Angeles,Headphones,Electronics,2,150,PayPal,300,489.5,402.288247,False
2,1003,2024-01-03,C003,Charlie,New York,Office Chair,Furniture,1,350,Credit Card,350,489.5,402.288247,False
3,1004,2024-01-03,C001,Alice,New York,Mouse,Electronics,3,25,Debit Card,75,489.5,402.288247,False
4,1005,2024-01-04,C004,Diana,Chicago,Desk,Furniture,1,500,Bank Transfer,500,489.5,402.288247,False
5,1006,2024-01-04,C005,Eve,Chicago,Laptop,Electronics,1,1100,Credit Card,1100,489.5,402.288247,False
6,1007,2024-01-05,C002,Bob,Los Angeles,Monitor,Electronics,2,300,Debit Card,600,489.5,402.288247,False
7,1008,2024-01-05,C003,Charlie,New York,Desk Lamp,Furniture,2,45,PayPal,90,489.5,402.288247,False
8,1009,2024-01-06,C006,Frank,Miami,Tablet,Electronics,1,600,Credit Card,600,489.5,402.288247,False
9,1010,2024-01-06,C001,Alice,New York,Keyboard,Electronics,1,80,Bank Transfer,80,489.5,402.288247,False
